In [531]:
import pandas as pd
import numpy as np
import glob
import requests

In [6]:
path = '/Users/niloofartehrani/Google Drive File Stream/My Drive/Programming/Data Incubator Challenge/house-office-expenditures-with-readme/'
csv_files = [file for file in glob.glob(path +'*detail*.csv') if not file.endswith('2015Q2-house-disburse-detail.csv')]

In [7]:
list_data = []
for filename in csv_files:
    data = pd.read_csv(filename,dtype = {"BIOGUIDE_ID": object},encoding = 'ISO-8859-1')
    list_data.append(data)
#data DataFrame is the main data before cleaning
data = pd.concat(list_data,ignore_index=True)

In [8]:
#cleaning the data
data_clean = data.copy()

In [9]:
#organizing the data for the 2017Q2 in the right columns
data_clean[data_clean['QUARTER']=='2017Q2'] = data[data['QUARTER']=='2017Q2'].loc[:,['PURPOSE','BIOGUIDE_ID',
          'CATEGORY','SORT SEQUENCE','START DATE','OFFICE','RECORDID','PROGRAM','END DATE','QUARTER','RECIP (orig.)',
          'TRANSCODE','BIOGUIDE_ID','PAYEE','DATE','TRANSCODELONG','AMOUNT']].values
#data_clean[data_clean['QUARTER']=='2017Q2']

In [10]:
data_clean.AMOUNT = pd.to_numeric(data_clean['AMOUNT'], errors = 'coerce').round(10)


In [11]:
#function for changing to datetime format more efficiently
def lookup(cl):
    """
    This is an extremely fast approach to datetime parsing.
    For large data, the same dates are often repeated. Rather than
    re-parse these, we store all unique dates, parse them, and
    use a lookup to convert all dates.
    """
    dates = {date:pd.to_datetime(date, errors = 'coerce') for date in cl.unique()}
    return cl.map(dates)

In [12]:
#converting to datetime format
data_clean['START DATE'] = lookup(data_clean['START DATE'])
data_clean['END DATE'] = lookup(data_clean['END DATE'])
data_clean['DATE'] = lookup(data_clean['DATE'])

In [13]:
#cleaning the YEAR column and changing the data type
data_clean.YEAR = data_clean.YEAR.str.extract('(\d+)',expand=True)
data_clean.YEAR = pd.to_numeric(data_clean['YEAR'], errors = 'coerce')
data_clean = data_clean[(data_clean['YEAR']>2007) | (data_clean['YEAR'].isnull())]

In [14]:
#finding the rows that have YEAR = NaN and replace them with the first 4 characters of QUARTER column if QUARTER
#is not null and has 6 characters
rep_VALUE_year = ((data_clean.loc[data_clean['YEAR'].isnull(),'QUARTER'].notnull())
                  & (data_clean.loc[data_clean['YEAR'].isnull(),'QUARTER'].str.len() == 6))
#find the index of the rows that should be replaced
rep_VALUE_index = data_clean[data_clean['YEAR'].isnull()].index[rep_VALUE_year].tolist()
#find the data to replace with
rep_VALUE_with = data_clean['QUARTER'][data_clean['YEAR'].isnull()][rep_VALUE_year].str[0:4].astype(int)
#replacing 
data_clean.loc[rep_VALUE_index,'YEAR'] = rep_VALUE_with

In [15]:
#1st cleaning of QUARTER by using the END DATE column
rep_quarter = data_clean.loc[data_clean['QUARTER'].str.len() <4]
rep_quarter_index = rep_quarter.loc[data_clean['END DATE'].notnull()].index.tolist()
data_clean.loc[rep_quarter_index, 'QUARTER'] = \
data_clean.loc[rep_quarter_index, 'END DATE'].dt.strftime('%Y') + data_clean.loc[rep_quarter_index, 'QUARTER']
#2nd cleaning of QUARTER by using the YEAR column
rep_quarter2 = data_clean.loc[data_clean['QUARTER'].str.len() <4]
rep_quarter_index2 = rep_quarter2.loc[data_clean['YEAR'].notnull()].index.tolist()
data_clean.loc[rep_quarter_index2, 'QUARTER'] = \
data_clean.loc[rep_quarter_index2, 'YEAR'].astype(int).astype(str) + data_clean.loc[rep_quarter_index2, 'QUARTER']
#3rd cleaning
#rep_quarter_index3 = data_clean['QUARTER'].loc[data_clean['QUARTER'].str.len() <4].index.tolist()
#data_clean['QUARTER'].loc[rep_quarter_index3[0]-2:rep_quarter_index3[-1]+2].replace(r'^Q',method = 'ffill',regex = True)

In [16]:
#cleaning the PAYEE column
data_clean['PAYEE'] = data_clean['PAYEE'].str.replace(',', '')
data_clean['PAYEE'] = data_clean['PAYEE'].str.replace('.', '')
data_clean['PAYEE'] = data_clean['PAYEE'].str.replace(' ', '')
data_clean['PAYEE'] = data_clean['PAYEE'].str.replace('BOLTONCAROLINEH', 'BOULTONCAROLINEH')


# ANSWERING THE QUESTIONS

#### What is the total of all the payments in the dataset?

In [19]:
pd.set_option('precision',10)

In [22]:
tot_payment = round(data_clean['AMOUNT'].sum(),10)
print('The total of all the payments is: {}'.format(tot_payment))

The total of all the payments is: 5570079451.790002


#### What was the average annual expenditure with a 'START DATE' date between January 1, 2010 and December 31, 2016 (inclusive)? Only consider payments with strictly positive amounts.

In [74]:
yearly_expense = data_clean.set_index('START DATE')
yearly_expense = yearly_expense.loc['January 1,2010':'December 31, 2016', 'AMOUNT']
yearly_expense = yearly_expense[yearly_expense>= 0]
annual_exp = yearly_expense.resample('A').mean()
mean_annual_exp = round(annual_exp.mean(),10)
print('the annual expenditure between January 1, 2010 and December 31, 2016 is: {}'.format(mean_annual_exp))

the annual expenditure between January 1, 2010 and December 31, 2016 is: 408.7948153241


#### What was the highest average staff salary among all representatives in 2016? Assume staff sizes is equal to the number of unique payees in the 'PERSONNEL COMPENSATION' category for each representative.

In [206]:
staff_salary_2016 = data_clean.loc[data_clean['START DATE'].dt.year == 2016,:]
staff_size_data = staff_salary_2016[staff_salary_2016['CATEGORY']=='PERSONNEL COMPENSATION']
staff_size = staff_size_data.groupby('BIOGUIDE_ID')['PAYEE'].nunique()
staff_salary_per_rep = staff_size_data.groupby(['BIOGUIDE_ID','PAYEE'])['AMOUNT'].sum()
max_staff_salary = round((staff_salary_per_rep/staff_size).max(),10)
print('The highest average staff salary among all representatives in 2016: {}'.format(max_staff_salary))

The highest average staff salary among all representatives in 2016: 3683.332


#### Define the 'COVERAGE PERIOD' for each payment as the difference (in days) between 'END DATE' and 'START DATE'. What is the standard deviation in 'COVERAGE PERIOD'? Only consider payments with strictly positive amounts.

In [25]:
coverage_data = data_clean.loc[data_clean['AMOUNT']>=0,['START DATE','END DATE']]
coverage_data['COVERAGE PERIOD'] = data_clean['END DATE'] - data_clean['START DATE']
coverage_std = coverage_data['COVERAGE PERIOD'].std()
coverage_std_sec = coverage_std.days*24*3600 + coverage_std.seconds + coverage_std.microseconds/10**6
coverage_std_days = round(coverage_std_sec/(3600*24),10)
print('The standard deviation in "COVERAGE PERIOD" is: {}'.format(coverage_std_days))

The standard deviation in "COVERAGE PERIOD" is: 58.3976096228


#### Find the 'OFFICE' with the highest total expenditures with a 'START DATE' in 2016. For this office, find the 'PURPOSE' that accounts for the highest total expenditures. What fraction of the total expenditures (all records, all offices) with a 'START DATE' in 2016 do these expenditures amount to?

In [71]:
office_data = data_clean.loc[data_clean['START DATE'].dt.year == 2016,:]
max_exp_officename = office_data.groupby('OFFICE')['AMOUNT'].sum().sort_values(ascending = False).index[0]
max_exp_office_data = office_data.loc[office_data['OFFICE'] == max_exp_officename,:]
max_exp_purpose = max_exp_office_data.groupby('PURPOSE')['AMOUNT'].sum().max()
tot_exp = office_data['AMOUNT'].sum()
exp_frac = round(max_exp_purpose/tot_exp,10)
print('The fraction is: {}'.format(exp_frac))

The fraction is: 0.0739107351


#### What was the median rate of annual turnover in staff between 2011 and 2016 (inclusive)? Turnover for 2011 should be calculated as the fraction of a representative's staff from 2010 who did not carry over to 2011. Only consider representatives who served for at least 4 years and had staff size of at least 5 every year that they served.

In [425]:
#cleaning some suspicious data
data_clean.loc[data_clean['START DATE'].dt.year==2061,'START DATE'] = data_clean.loc[data_clean['START DATE'].dt.year==2061,'START DATE']- pd.DateOffset(years=50)
data_clean.loc[data_clean['START DATE'].dt.year==2051,'START DATE'] = data_clean.loc[data_clean['START DATE'].dt.year==2051,'START DATE']- pd.DateOffset(years=36)
data_clean.loc[data_clean['START DATE'].dt.year==2044,'START DATE'] = data_clean.loc[data_clean['START DATE'].dt.year==2044,'START DATE']- pd.DateOffset(years=30)

In [427]:
#I assumed that the 4 year serving condition and the staff size of 5 is between applied to the years between 2011 and 2016
rep_data = data_clean.groupby(data_clean['START DATE'].dt.year)['BIOGUIDE_ID'].unique()

years = list(range(2011,2017))
idx = [i for i in data_clean['BIOGUIDE_ID'].unique() if str(i) != 'nan']

rep_serv_year = pd.DataFrame(0, index=idx, columns= years)
for y in sorted(years):
    for i in range(len(rep_serv_year)):
        if rep_serv_year.index[i] in rep_data[y]:
            rep_serv_year.loc[rep_serv_year.index[i],y] = 1

In [428]:
rep_staff_size_data = data_clean[data_clean['CATEGORY']=='PERSONNEL COMPENSATION']\
                        .groupby([data_clean['START DATE'].dt.year,'BIOGUIDE_ID'])['PAYEE'].nunique()

idx2 = rep_staff_size_data.index.get_level_values(1).unique()
rep_staff_size = pd.DataFrame(0, index=idx2, columns= years)

for y in years:
    for i in rep_staff_size_data.loc[y].index:
        if rep_staff_size_data.loc[y,i]>=5:
            rep_staff_size.loc[i,y] = True


In [429]:
reps = rep_staff_size.index.intersection(rep_serv_year.index)
conditions = pd.DataFrame()
conditions['total years served'] = rep_serv_year.loc[reps,years].sum(axis =1)
conditions['#years of staff size = 5'] = rep_staff_size.loc[reps,years].sum(axis =1)
conditions['conditions met'] = (conditions['total years served'] == conditions['#years of staff size = 5']) & \
                            (conditions['total years served'] >= 4)
reps = conditions[conditions['conditions met'] == True].index

In [455]:
turnover_data = data_clean.loc[(data_clean['START DATE'].dt.year >= 2010) \
                             & (data_clean['START DATE'].dt.year <= 2016) \
                             & (data_clean['BIOGUIDE_ID'].isin(list(reps)))\
                             & (data_clean['CATEGORY']=='PERSONNEL COMPENSATION'),:]
turnover_data = turnover_data.dropna(subset = ['PAYEE'])
turnover_per_year_reps = pd.Series(index = years)
for y in range(2010,2016):
    year1_staff = turnover_data.loc[turnover_data['START DATE'].dt.year == y].groupby('BIOGUIDE_ID')['PAYEE'].unique()
    year2_staff = turnover_data.loc[turnover_data['START DATE'].dt.year == y+1].groupby('BIOGUIDE_ID')['PAYEE'].unique()
    staff_list_year1 = []
    staff_list_year2 = []
    for rep in reps:
        if (rep in year1_staff.keys()) and (rep in year2_staff.keys()): 
            staff_list_year1 += list(year1_staff[rep])
            staff_list_year2 += list(year2_staff[rep])
    turnover_frac_rep = (len(set(staff_list_year1)) - len(set(staff_list_year1).intersection(set(staff_list_year2))))\
                             /(len(set(staff_list_year1)))
    turnover_per_year_reps[y+1] = turnover_frac_rep                    
        

median_turnover_rate = round(turnover_per_year_reps.median(),10)
print('the median rate of annual turnover in staff between 2011 and 2016 is: {}'.format(median_turnover_rate))

the median rate of annual turnover in staff between 2011 and 2016 is: 0.2354594634


#### What percentage of the expenditures of the top 20 spenders in 2016 come from members of the Democratic Party? Representatives are identified by their 'BIOGUIDE_ID', which can be used to look up representatives with ProPublica's Congress API to find their party affiliation. Consider an expenditure as being in 2016 if its 'START DATE' is in 2016.

In [561]:
top_spenders_2016 = data_clean.loc[data_clean['START DATE'].dt.year == 2016]
top_spenders_2016 = top_spenders_2016.groupby('BIOGUIDE_ID')['AMOUNT'].sum()
top_spenders_2016 = top_spenders_2016.sort_values(ascending = False)[0:20]
top_spenders = top_spenders_2016.keys()
reps_parties = pd.DataFrame(top_spenders_2016)

In [569]:
for rep in list(top_spenders):
    url = "https://api.propublica.org/congress/v1/members/{}.json".format(rep)
    r = requests.get(url, headers={'X-API-Key':'1yk2z0GoZu7Kvcst1UobxPrrCV3fBParGwOVxsnZ'})
    json_data = r.json()
    member = json_data['results'][0]['member_id']
    party =  json_data['results'][0]['roles'][0]['party']
    reps_parties.loc[member,'party'] = party

democrat_spender_perc = ((reps_parties['AMOUNT'][reps_parties['party'] == 'D'].sum())/(reps_parties['AMOUNT'].sum()))*100
print('The percentage of the expenditures of the top 20 spenders in 2016 is: {}'.format(round(democrat_spender_perc,10)))

The percentage of the expenditures of the top 20 spenders in 2016 is: 33.4165808451
